In [13]:
import attr
import astropy.units as u
import numpy as np

In [30]:
@attr.s
class galaxy:
    x = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    #y = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    
    def __change_units_to_array__(self,*args,**kwargs):
        def new_func(self,*args,**kwargs):    
            self.x = self.x.to_value(u.kpc)
            #self.y = self.y.to(u.M_sun)
            return self
        return new_func
    
    @__change_units_to_array__
    def func(self):
        return self

In [31]:
yy = galaxy(x=5*u.Mpc)
yy

galaxy(x=<Quantity 5. Mpc>)

In [32]:
yy.x, 

(<Quantity 5. Mpc>,)

In [33]:
yy.func()

galaxy(x=5000.0)

In [34]:
type(yy.x)

numpy.float64

In [15]:
5*u.Mpc + 300*u.kpc

<Quantity 5.3 Mpc>

In [16]:
import numpy as np

np.asarray(y.func())

array(5000.)

In [17]:
type(np.asarray(y.func()))

numpy.ndarray

In [44]:
type(y.x)

astropy.units.quantity.Quantity

In [43]:
y.x

<Quantity 5000. kpc>

In [25]:
u.ma

AttributeError: module 'astropy.units' has no attribute 'mass'

In [40]:
@u.quantity_input(a=['length', 'speed'])
def myfunction(a):
    return a.to_value(u.kpc)

In [41]:
myfunction(5*u.Mpc)

5000.0

In [85]:
@attr.s
class galaxy:
    x = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    y = attr.ib(validator=attr.validators.instance_of(u.Quantity))
    
    @u.quantity_input(a=['length'])
    def myfunction(self):
        return self.a.to_value(u.kpc)
    
    @u.quantity_input(a=['mass'])
    def myfunction2(self):
        return self.y.to_value(u.Msun)



In [86]:
y=galaxy(y=3e20*u.m,x=5*u.Mpc)

In [100]:
y.x.unit.physical_type


'length'

In [88]:
y.myfunction()

AttributeError: 'galaxy' object has no attribute 'a'

In [78]:
y.y

<Quantity 3.e+20 m>

In [103]:
y = 3*u.M_sun

In [104]:
y.unit.physical_type

'mass'